In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
import prometheux_chain as pmtx

PMTX_TOKEN = "dev-token"
os.environ["PMTX_TOKEN"] = PMTX_TOKEN

#### Project functions

In [ ]:
#pmtx.cleanup_projects()

In [ ]:
project_id = pmtx.save_project(project_name="test_project")
project_id

In [ ]:
pmtx.list_projects()

In [ ]:
project = pmtx.load_project(project_id=project_id)
project

#### Data functions

In [ ]:
#pmtx.cleanup_sources()

In [ ]:
db = pmtx.data.database.Database(
    database_type="csv",
    username="",
    password="",
    host=os.getcwd() + "/../resources/ownership",
    port=0000,
    database_name="ownership.csv",
    options={} #{"delimiter": "\t"}
)
pmtx.connect_sources(database_payload=db)

In [ ]:
pmtx.list_sources()

In [ ]:
pmtx.infer_schema(database=db, add_bind=True, add_model=False)

#### Concept functions

In [ ]:
#pmtx.cleanup_concepts(project_id=project_id)

In [ ]:
concept_logic = """
ownership(Owner,Company,Shares) :- ownership_csv(Owner,Company,Shares).
@output("ownership").
"""

pmtx.save_concept(project_id=project_id, concept_logic=concept_logic)

concept_logic = """
owner(Owner) :- ownership(Owner,Company,Shares).
@output("owner").
"""

pmtx.save_concept(project_id=project_id, concept_logic=concept_logic)

concept_logic = """
company("Apple", "Redwood City, CA").
company("Google", "Mountain View, CA").
company("Microsoft", "Redmond, WA").
company("Amazon", "Seattle, WA").
company("Facebook", "Menlo Park, CA").
company("Twitter", "San Francisco, CA").
company("LinkedIn", "Sunnyvale, CA").
company("Instagram", "Menlo Park, CA").

location(Location) :- company(_,Location).

@output("location").
"""

pmtx.save_concept(project_id=project_id, concept_logic=concept_logic)

In [ ]:
pmtx.run_concept(project_id=project_id, concept_name="ownership", persist_outputs=True)

In [ ]:
pmtx.run_concept(project_id=project_id, concept_name="location", persist_outputs=False)

In [ ]:
pmtx.list_concepts(project_id=project_id)

##### KG functions

In [ ]:
pmtx.save_kg(project_id=project_id, concepts=["owner", "location"])

In [ ]:
pmtx.load_kg(project_id=project_id)

#### GraphRAG function

In [ ]:
graph_rag_result = pmtx.graph_rag(project_id=project_id,
                                  question="This is the question", 
                                  rag_concepts=[{"concept": "ownership", "field_to_embed": "Owner"}],
                                  graph_selected_concepts=["owner"])

graph_rag_result['graph_rag_results']